In [1]:
%%javascript

IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

In [2]:
cd ../../

/home/pnickl/pycharm-projects/Healing-POEs-ICML/Code/bayesian_benchmarks_modular


In [3]:

from matplotlib import pyplot
import numpy as np
import pandas
from matplotlib import pyplot as plt
from scipy.stats import rankdata
from IPython.display import display, HTML

from bayesian_benchmarks.database_utils import Database
from bayesian_benchmarks.data import  _ALL_REGRESSION_DATATSETS
from bayesian_benchmarks.data import  _ALL_CLASSIFICATION_DATATSETS
ALL_DATATSETS = {}
ALL_DATATSETS.update(_ALL_REGRESSION_DATATSETS)
ALL_DATATSETS.update(_ALL_CLASSIFICATION_DATATSETS)
from bayesian_benchmarks.data import regression_datasets
from bayesian_benchmarks.data import classification_datasets


2022-08-25 14:04:35.936125: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


1


2022-08-25 14:04:36.956223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 14:04:36.965860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 14:04:36.966350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [38]:
def rankarray(A):
    ranks = []
    for a in A:
        ranks.append(rankdata(a))
    return np.array(ranks)


def read_regression_classification(fs, models_names, datasets, task,file='results.db'):
    if task == 'classification':
        fields = ['dataset', 'N', 'D', 'K'] + [m[1] for m in models_names]
    else:
        fields = ['dataset', 'N', 'D'] + [m[1] for m in models_names]

    results = {}
    for f in fs:
        results[f] = {'table':{f:[] for f in fields}, 'vals':[], 'vals_std':[]}

    with Database('bayesian_benchmarks/results/'+file) as db:

        for dataset in datasets:
            if dataset != 'wam':
                for f in fs:
                    results[f]['table']['dataset'].append(dataset[:10])
                    results[f]['table']['N'].append(ALL_DATATSETS[dataset].N)
                    results[f]['table']['D'].append(ALL_DATATSETS[dataset].D)
                    if task == 'classification':
                        results[f]['table']['K'].append(ALL_DATATSETS[dataset].K)

                row_std = {f:[] for f in fs}
                row = {f:[] for f in fs}
                for model, name in models_names:
                    res = db.read(task, fs, {'model':model,
                                             'dataset':dataset})
                    if len(res) == 0:
                        for f in fs:
                            results[f]['table'][name].append('')
                            row[f].append(np.nan)
                    else:
                        for i, f in enumerate(fs):
                            if f=='test_loglik':
                                L = [-float(l[i]) for l in res]
                            else:
                                L = [float(l[i]) for l in res]
                            m = np.average(L)
                            std = np.std(L) if len(L) > 1 else np.nan
                            if m < 1000 and m > -1000:
                                r = '{:.10f}({:.10f})'.format(m, std)
                                row[f].append(m)
                                row_std[f].append(std)
                            else:
                                r = 'nan'
                                row[f].append(np.nan)

                            # stderr = np.std(L)/float(len(L))**0.5
                            # r = '{:.3f} ({:.3f})'.format(m, stderr)

                            results[f]['table'][name].append(r)
                for f in fs:
                    results[f]['vals'].append(row[f])
                    results[f]['vals_std'].append(row_std[f])

        for f in fs:
            #if 'unnormalized' not in f:
            vals = np.array(results[f]['vals'])
            vals_std = np.array(results[f]['vals_std'])

            stds = np.nanmean(vals_std, 0)
            avgs = np.nanmean(vals, 0)
            meds = np.nanmedian(vals, 0)
            rks = np.nanmean(rankarray(vals), 0)

            for s, n in [[avgs, 'avg'], [stds, 'std'], [meds, 'median'], [rks, 'avg rank']]:
                results[f]['table']['dataset'].append(n)
                results[f]['table']['N'].append('')
                results[f]['table']['D'].append('')
                if task == 'classification':
                    results[f]['table']['K'].append('')
                for ss, name in zip(s, [m[1] for m in models_names]):
                    results[f]['table'][name].append('{:.10f}'.format(ss))
    
    return results, fields

In [39]:
# models_names = [
#                ['gPoE_100_100_clustering_variance', 'rBCM/gPoE_var'],
#                ['bar_100_100_clustering_variance', 'bar_var'],
#                ['rBCM_100_100_clustering_diff_entr', 'rbcm_entr'],
#                ['gp', 'gp'],
#                ['linear', 'linear']]


models_names = [
                # ['rBCM_100_100_clustering_variance', 'rbcm_var'],
                # ['rBCM_100_100_clustering_diff_entr', 'rBCM_entr'], # compare to results in paper
                # ['gPoE_100_100_clustering_variance', 'rBCM/gPOE_var'], # compare to results in paper
                ['linear', 'linear'], # compare to results in paper
                ['PoE_100_100_clustering_no_weights', 'PoE (Ng & Deisenroth)' ],
                ['gPoE_100_100_clustering_diff_entr', 'gPoE_entr (Cao & Fleet)']
]


# fs = 'test_loglik', 'test_rmse', 'test_loglik_unnormalized', 'test_rmse_unnormalized'
fs = 'test_loglik', 'test_rmse', 'test_loglik_unnormalized', 'test_rmse_unnormalized', 'test_mse_sklearn', 'test_mse_unnormalized_sklearn', 'test_nmse_sklearn', 'test_nmse_unnormalized_sklearn'

results, fields = read_regression_classification(fs, models_names, regression_datasets, 'regression')


In [40]:
# print('NLPDSd')
# display(HTML(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_html(index=False)))
#
# print('normalised test rmse')
# display(HTML(pandas.DataFrame(results['test_rmse']['table'], columns=fields).to_html(index=False)))
#
# print('unnormalised test rmse')
# display(HTML(pandas.DataFrame(results['test_rmse_unnormalized']['table'], columns=fields).to_html(index=False)))
#
# print('sklearn: normalised test mse')
# display(HTML(pandas.DataFrame(results['test_mse_sklearn']['table'], columns=fields).to_html(index=False)))

print('sklearn: unnormalised test mse')
display(HTML(pandas.DataFrame(results['test_mse_unnormalized_sklearn']['table'], columns=fields).to_html(index=False)))

print('sklearn: unnormalized test nmse')
display(HTML(pandas.DataFrame(results['test_nmse_unnormalized_sklearn']['table'], columns=fields).to_html(index=False)))


sklearn: unnormalised test mse


dataset,N,D,linear,PoE (Ng & Deisenroth),gPoE_entr (Cao & Fleet)
sarcos0,48933,21,31.0857592861(0.0000000000),56.1261689388(1.3619717942),2.9701203647(0.0518372946)
sarcos1,48933,21,22.6845575348(0.0000000000),13.2338561923(0.4973648882),0.5394248982(0.0131459492)
sarcos2,48933,21,9.0841228118(0.0000000000),3.6051866244(0.1229947051),0.1789287140(0.0044620920)
sarcos3,48933,21,9.7322006864(0.0000000000),2.3643932817(0.0756264099),0.2265600857(0.0047684772)
sarcos4,48933,21,0.1340376345(0.0000000000),0.1055203532(0.0039162828),0.0035983113(0.0001439037)
sarcos5,48933,21,0.8274044870(0.0000000000),0.3117010379(0.0096772373),0.0081096160(0.0001856999)
sarcos6,48933,21,0.4364194860(0.0000000000),0.2560262553(0.0059206543),0.0198398965(0.0003353767)
avg,,,10.5692145609,10.8575503834,0.5637974123
std,,,0.0000000000,0.2967817103,0.0106969705
median,,,9.0841228118,2.3643932817,0.1789287140


sklearn: unnormalized test nmse


dataset,N,D,linear,PoE (Ng & Deisenroth),gPoE_entr (Cao & Fleet)
sarcos0,48933,21,0.0750541817(0.0000000000),0.1355123304(0.0032883764),0.0071711278(0.0001251572)
sarcos1,48933,21,0.1021822311(0.0000000000),0.0596116962(0.0022403723),0.0024298309(0.0000592157)
sarcos2,48933,21,0.0918075180(0.0000000000),0.0364353546(0.0012430302),0.0018083200(0.0000450956)
sarcos3,48933,21,0.0514072291(0.0000000000),0.0124891493(0.0003994723),0.0011967310(0.0000251880)
sarcos4,48933,21,0.1416681356(0.0000000000),0.1115274211(0.0041392291),0.0038031561(0.0001520958)
sarcos5,48933,21,0.2816991827(0.0000000000),0.1061221313(0.0032947245),0.0027610102(0.0000632236)
sarcos6,48933,21,0.0648678247(0.0000000000),0.0380548229(0.0008800248),0.0029489309(0.0000498492)
avg,,,0.1155266147,0.0713932723,0.0031598724
std,,,0.0000000000,0.0022121756,0.0000742607
median,,,0.0918075180,0.0596116962,0.0027610102


In [ ]:
# models_names = [
#                ['bar_10_500_random_variance', 'bar_var'],
#                ['gPoE_10_500_random_variance', 'gpoe_var'],
#                ['gPoE_10_500_random_uniform', 'gpoe_unif'],
#                ['linear', 'linear']]
#
#
# fs = 'top_1_acc', 'top_2_acc', 'top_3_acc', 'test_loglik'
#
# results, fields = read_regression_classification(fs, models_names, classification_datasets, 'classification')


In [ ]:

# print('top 1 accuracy')
# display(HTML(pandas.DataFrame(results['top_1_acc']['table'], columns=fields).to_html(index=False)))
#
#
# print('top 2 accuracy')
# display(HTML(pandas.DataFrame(results['top_2_acc']['table'], columns=fields).to_html(index=False)))
#
# print('top 3 accuracy')
# display(HTML(pandas.DataFrame(results['top_3_acc']['table'], columns=fields).to_html(index=False)))
#
# print('NLPDs')
# display(HTML(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_html(index=False)))
